In [6]:
import requests
import json
from configparser import ConfigParser, ExtendedInterpolation
from urllib.parse import urlparse
import time
import base64
import os

# Configura o certificado Petrobras para a lib requests
PTB_CERT_PATH = os.path.join(os.path.abspath(''), '../petrobras-ca-root.pem')
os.environ['REQUESTS_CA_BUNDLE'] = PTB_CERT_PATH

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../config.ini', 'UTF-8')

base_url = "https://apid.petrobras.com.br/ia/texto/v1/azure-ai-document-intelligence"

default_headers = {
    "Content-Type": "application/json",
    "apikey": config['HUB']['apikey_modelos_texto'],
}

def consultar_resultado(url, wait_time_seconds=3, max_retries=3):

    parsed_url = urlparse(url)
    
    print(f'Consultando resultado do path {parsed_url.path}')
    
    for i in range(max_retries):
        time.sleep(wait_time_seconds)

        response = requests.get(
            f'{base_url}{parsed_url.path}?{parsed_url.query}',
            headers=default_headers
        )
        
        body = response.json()
        if body.get('status', '') == 'succeeded':
            return body
        else:
            print(f'Consulta de resultado retornou: {json.dumps(body, indent=4)}')


def analisar_documento(pdf_base64):

    response = requests.post(
        f"{base_url}/formrecognizer/documentModels/prebuilt-document:analyze?api-version=2023-07-31",
        headers=default_headers,
        json={
            #"urlSource": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"
            "base64Source": pdf_base64
        }
    )

    if response.status_code != 202:
        print(f'Erro na requisição ({response.status_code}): {json.dumps(response.json(), indent=4)}')
    else:
        
        resultado = consultar_resultado(response.headers['Operation-Location'])
        print(f'Resultado: {json.dumps(resultado, indent=4)}')
        

with open("sample-layout.pdf", "rb") as f:
    pdf_data = f.read()

analisar_documento(base64.b64encode(pdf_data).decode('utf-8'))



Consultando resultado do path /formrecognizer/documentModels/prebuilt-document/analyzeResults/04fb740f-1501-45d0-ab50-1e4ea5cff418
Consulta de resultado retornou: {
    "status": "running",
    "createdDateTime": "2024-04-17T12:59:03Z",
    "lastUpdatedDateTime": "2024-04-17T12:59:03Z"
}
Resultado: {
    "status": "succeeded",
    "createdDateTime": "2024-04-17T12:59:03Z",
    "lastUpdatedDateTime": "2024-04-17T12:59:09Z",
    "analyzeResult": {
        "apiVersion": "2023-07-31",
        "modelId": "prebuilt-document",
        "stringIndexType": "textElements",
        "content": "UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549\nFORM 10-Q :selected: \u2612 QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the Quarterly Period Ended March 31, 2020\nOR :unselected:\n\u2610 TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the Transition Period From\nto\nCommission File Number 0